In [1]:
      
from dask.distributed import Client
import dask.array as da


import skimage as ski
import numpy as np
import dask
import glob
import platform

In [2]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import time

cluster = SLURMCluster(
    cores=1,                      # Number of cores per job
    memory="64GB",                # Memory per job
    queue="gpu",         # Queue/partition name
    walltime="22:00:00",           # Job time limit
    local_directory="$TMPDIR",     # Temporary directory (optional)
    log_directory="logs",          # Directory for log files (optional)
    job_extra_directives =[
        '--gres=gpu:rtx4500:1',  # Number of GPUs per job
        '--qos=rtx4500', # Quality of Service for GPU jobs
    ],
)

client = Client(cluster)
print(client)

cluster.scale(jobs=6)  # Request 4 jobs
client.cluster
client

<Client: 'tcp://10.0.53.5:42839' processes=0 threads=0, memory=0 B>


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.0.53.5:8787/status,
Dashboard: http://10.0.53.5:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.53.5:42839,Workers: 0
Dashboard: http://10.0.53.5:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B



array = da.random.random((20000, 20000,200), chunks=(1000, 1000,100))
result = array.mean().compute() # This will be executed across your 5 PCs!

print(f"The mean of the large array is: {result}")

In [6]:
def process_image(fname):
    # If this system is on linux replace the fname
    # with the full path to the image file.
    # For example: fname = '/home/user/images/image.tif'

    import cellpose.models as models
    model = models.CellposeModel(gpu=True)
    img = ski.io.imread(fname)
    rslts = model.eval(img, channel_axis=None, z_axis=0, stitch_threshold=0.5)[0]
    ski.io.imsave(fname+'f', rslts)
    return 0

In [7]:
#fnames = glob.glob('\\\\ion\\projects/smc/public/SMC/dask/data/*.tif')[0:10] * 5
fnames = glob.glob('/n/projects/smc/public/SMC/dask/data/*.tif')[0:10] * 5
fnames

['/n/projects/smc/public/SMC/dask/data/img4.tif',
 '/n/projects/smc/public/SMC/dask/data/img2.tif',
 '/n/projects/smc/public/SMC/dask/data/img6.tif',
 '/n/projects/smc/public/SMC/dask/data/img1.tif',
 '/n/projects/smc/public/SMC/dask/data/img5.tif',
 '/n/projects/smc/public/SMC/dask/data/img0.tif',
 '/n/projects/smc/public/SMC/dask/data/img3.tif',
 '/n/projects/smc/public/SMC/dask/data/img4.tif',
 '/n/projects/smc/public/SMC/dask/data/img2.tif',
 '/n/projects/smc/public/SMC/dask/data/img6.tif',
 '/n/projects/smc/public/SMC/dask/data/img1.tif',
 '/n/projects/smc/public/SMC/dask/data/img5.tif',
 '/n/projects/smc/public/SMC/dask/data/img0.tif',
 '/n/projects/smc/public/SMC/dask/data/img3.tif',
 '/n/projects/smc/public/SMC/dask/data/img4.tif',
 '/n/projects/smc/public/SMC/dask/data/img2.tif',
 '/n/projects/smc/public/SMC/dask/data/img6.tif',
 '/n/projects/smc/public/SMC/dask/data/img1.tif',
 '/n/projects/smc/public/SMC/dask/data/img5.tif',
 '/n/projects/smc/public/SMC/dask/data/img0.tif',


In [8]:
rslts = [dask.delayed(process_image)(fname) for fname in fnames]
# Compute the results in parallel
dask.compute(*rslts)

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0)

In [9]:
# When you're done, you can close the connection
client.close()
cluster.close()
